In [1]:
from decadal_stats import collect_data

In [2]:
collect_data([1961,1964],[300,320])

[array([ 28.82209659,  29.75720736,  27.67574181,  31.40685213,
         25.12879964,  24.55137525,  27.21288713,  24.08732992,
         26.26504251,  36.26419439,  22.87906306,  30.20074631,
         30.37401985,  28.22128966,  28.24600119,  25.92663933,
         25.24226683,  24.70526882,  32.2638267 ]),
 array([ 12.65920864,  18.73594929,  16.6198177 ,  21.04861813,
         31.57616516,  33.92386494,  26.83138114,  12.74176862,
         24.89612898,   8.35879039,  24.6422515 ,  14.56655946,
         19.29091215,  10.13163313,  13.61999525,  20.5084591 ,
         20.11551487,  12.96779755,   9.01343167])]

In [3]:
results = {'latitude':{},'intensity':{},'frequency':{}}

In [5]:
results['latitude']

{}